In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

df = pd.read_excel("Medicine data 100 conditions.xlsx")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 47 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   CONDITION                           102 non-null    object 
 1   sym 1                               101 non-null    object 
 2   sym 2                               91 non-null     object 
 3   sym 3                               91 non-null     object 
 4   sym 4                               68 non-null     object 
 5   Test 1                              98 non-null     object 
 6   Test 2                              73 non-null     object 
 7   Test 3                              47 non-null     object 
 8   Test 4                              27 non-null     object 
 9   drug class                          98 non-null     object 
 10  BRAND_NAME                          95 non-null     object 
 11  api strength unit - mg or ml or     91 non-nu

In [4]:
pres = ['BRAND_NAME']
for prescribed in pres:
    df[pres] = df[pres].fillna('')

Combining Relevant Features into a Single Feature

In [5]:
df['prescription'] = df[pres].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)
df['prescription']

0         Cheston Cold
1              Lamisil
2              Zovirax
3              Orencia
4             Relenza®
            ...       
97           Dramamine
98          Dailyshine
99     Cantharone plus
100          Zithromax
101                   
Name: prescription, Length: 102, dtype: object

In [6]:
df['medicine'] = df.apply(lambda x: list([x['CONDITION'],x['prescription'].strip()]),axis=1)
df['medicine'].head()

0    [Common Cold, Cheston Cold]
1      [Athlete's Foot, Lamisil]
2         [Chicken pox, Zovirax]
3           [Arthritis, Orencia]
4          [Influenza, Relenza®]
Name: medicine, dtype: object

In [7]:
da = list(df['medicine'])
da

[['Common Cold', 'Cheston Cold'],
 ["Athlete's Foot", 'Lamisil'],
 ['Chicken pox', 'Zovirax'],
 ['Arthritis', 'Orencia'],
 ['Influenza', 'Relenza®'],
 ['Vitiligo ', 'Protopic; Elidel'],
 ['COPD', 'Asthalin respules'],
 ['Aspergillosis (Invasive)', 'VFEND'],
 ['Covid 19', 'Pan 40'],
 ['Acne', ''],
 ['Acute sinusitis', ''],
 ['Allergies', 'Allegra-m'],
 ["Alzheimer's and other Dementias", 'Aricept'],
 ['Iron deficiency Anaemia', 'livogen z'],
 ['Megaloblastic anemia(Vit.B12 and folic acid)', '2B12'],
 ['Hemolytic anemia', 'PredniSONE'],
 ['Thallasemia', 'Ferokos-z'],
 ['Anal Itching', 'hydrocortisone'],
 ['Anemia', ''],
 ['Anxiety', 'lexapro 10'],
 ['Arthritis', 'Plaquenil'],
 ['Autism', 'Respidon 2'],
 ['Bad breath ', 'cepacol'],
 ['Conjunctivitis', 'Chloromycetin'],
 ['Constipation', 'MiraLAX'],
 ['Contraception', 'Durex'],
 ['Dementia', 'exelon'],
 ['depression', 'prozac'],
 ['diabetes', 'glyciphage'],
 ['Ear infections', 'Amoxill'],
 ['Eczema', 'Framycetin'],
 ['Edema', 'Lasix'],
 ['

In [8]:
te = TransactionEncoder()
te_ary = te.fit(da).transform(da,sparse = False)
trans_df = pd.DataFrame(te_ary, columns=te.columns_)
trans_df.head()

,,2B12,Abilify,Acne,"Actigall, Urso",Acute sinusitis,Allegra-m,Allergies,Alzheimer's and other Dementias,Amoxill,...,systemic lupus erythomatous,temodar,"tylenol,aleve,",typhoid,uremia,urinary calculi,urticaria,vaginal cancer,whooping cough,zyrtec
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
def convert_into_binary(x):
    if x == True:
        return 1
    else:
        return 0
    
trans_df = trans_df.applymap(convert_into_binary)

trans_df.head()

,,2B12,Abilify,Acne,"Actigall, Urso",Acute sinusitis,Allegra-m,Allergies,Alzheimer's and other Dementias,Amoxill,...,systemic lupus erythomatous,temodar,"tylenol,aleve,",typhoid,uremia,urinary calculi,urticaria,vaginal cancer,whooping cough,zyrtec
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#call apriori function and pass minimum support here we are passing 3%. 
# means 3 times in total number of transaction that item was present.

frequent_itemsets = apriori(trans_df, min_support=0.001, use_colnames=True)

In [11]:
# We have association rules which need to put on frequent itemset
# setting threshold with lift 
rules_mlxtend = association_rules(frequent_itemsets, metric = 'lift', min_threshold=10)
rules_mlxtend

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(),(Acne),0.068627,0.009804,0.009804,0.142857,14.571429,0.009131,1.155229
1,(Acne),(),0.009804,0.068627,0.009804,1.000000,14.571429,0.009131,inf
2,(),(Acute sinusitis),0.068627,0.009804,0.009804,0.142857,14.571429,0.009131,1.155229
3,(Acute sinusitis),(),0.009804,0.068627,0.009804,1.000000,14.571429,0.009131,inf
4,(),(Anemia),0.068627,0.009804,0.009804,0.142857,14.571429,0.009131,1.155229
...,...,...,...,...,...,...,...,...,...
199,(gamaSTAN),(measles),0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf
200,(malaria),(malarone),0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf
201,(malarone),(malaria),0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf
202,(melanoma),(temodar),0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf


In [12]:
# Get maximum confidence and lift
print(rules_mlxtend['confidence'][rules_mlxtend['confidence']== max(rules_mlxtend['confidence'])])
print(rules_mlxtend['lift'][rules_mlxtend['lift']==max(rules_mlxtend['lift'])])

1      1.0
3      1.0
5      1.0
7      1.0
9      1.0
      ... 
199    1.0
200    1.0
201    1.0
202    1.0
203    1.0
Name: confidence, Length: 187, dtype: float64
14     102.0
15     102.0
16     102.0
17     102.0
18     102.0
       ...  
199    102.0
200    102.0
201    102.0
202    102.0
203    102.0
Name: lift, Length: 170, dtype: float64


In [13]:
#Convert frozenset objects into string
rules_mlxtend["antecedents"] = rules_mlxtend["antecedents"].apply(lambda x: ', '.join(list(x))).astype(str)
rules_mlxtend["consequents"] = rules_mlxtend["consequents"].apply(lambda x: ', '.join(list(x))).astype(str)
rules_mlxtend

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,,Acne,0.068627,0.009804,0.009804,0.142857,14.571429,0.009131,1.155229
1,Acne,,0.009804,0.068627,0.009804,1.000000,14.571429,0.009131,inf
2,,Acute sinusitis,0.068627,0.009804,0.009804,0.142857,14.571429,0.009131,1.155229
3,Acute sinusitis,,0.009804,0.068627,0.009804,1.000000,14.571429,0.009131,inf
4,,Anemia,0.068627,0.009804,0.009804,0.142857,14.571429,0.009131,1.155229
...,...,...,...,...,...,...,...,...,...
199,gamaSTAN,measles,0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf
200,malaria,malarone,0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf
201,malarone,malaria,0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf
202,melanoma,temodar,0.009804,0.009804,0.009804,1.000000,102.000000,0.009708,inf


In [14]:
pred_med = []
condition = df.CONDITION.tolist()

for x in condition:
    diagnosis = [x]
    rec = ['consequents','lift']
    med = rules_mlxtend[rec].loc[(rules_mlxtend['antecedents'].isin(diagnosis))]
    med = med.sort_values(by=['lift']).reset_index()
    med = med[~med['consequents'].isin(condition)]
    med = med['consequents'][0]
    pred_med.append(med)

In [15]:
df['predmed'] = pd.Series(pred_med)

def try_join(l):
    try:
        return ','.join(map(str, l))
    except TypeError:
        return np.nan

#df['predmedstr'] = [try_join(l) for l in df['predmed']]
x = ['predmed','prescription']
y = df[x]
y.count()

predmed         102
prescription    102
dtype: int64

In [16]:
df.head()

,CONDITION,sym 1,sym 2,sym 3,sym 4,Test 1,Test 2,Test 3,Test 4,drug class,...,Unnamed: 40,note 1,note 2,note 3,note 4,Unnamed: 45,Unnamed: 46,prescription,medicine,predmed
0,Common Cold,"sneezing, watery eyes or itchy/watery nose and...",Fever (although most people with colds do not ...,Pain in the muscles,NaN,Full body checkup,Blood Test,NaN,NaN,NaN,...,NaN,At least a gap of 4-6 hours should be there be...,exercise for 30 minutes daily,rest\n,NaN,NaN,NaN,Cheston Cold,"[Common Cold, Cheston Cold]",Cheston Cold
1,Athlete's Foot,"scaly rash that usually causes itching, stingi...","Moist, raw skin between the toes","Inflamed skin that might appear reddish, purpl...",NaN,Skin lesion Potassium Hydroxide exam,NaN,NaN,NaN,Antifungal,...,NaN,"Use it regularly(1-2 weeks),Once all signs of ...",Wash your feet everyday and dry them,Choose footwear that can help your feet stay dry,NaN,NaN,NaN,Lamisil,"[Athlete's Foot, Lamisil]",Lamisil
2,Chicken pox,Raised pink/red bumps (papules)which burst out...,"Fever, Loss of appetite, Headache, Tiredness","Skin: blister, scab, ulcers, or red spots",NaN,blood tests or a culture of lesion samples.,NaN,NaN,NaN,Anti - Viral,...,NaN,The dosage should be reduced in patients with ...,NaN,NaN,NaN,NaN,NaN,Zovirax,"[Chicken pox, Zovirax]",Zovirax
3,Arthritis,"Pain areas: in the joints, ankle, back, finger...",Hand: bump on the finger or bony outgrowth in ...,"Also common: flare, physical deformity, rednes...",NaN,X-ray,Arthroscopy,Ultrasound,NaN,anti-inflammatory,...,NaN,Self-care,Exercises can improve range of motion and stre...,NaN,NaN,NaN,NaN,Orencia,"[Arthritis, Orencia]",Orencia
4,Influenza,"fever, chills, muscle aches, cough, congestion...",NaN,"body chills, dehydration, fatigue, fever, flus...",NaN,Viral culture tests,reverse transcription polymerase chain reactio...,NaN,NaN,Anti influenza drugs,...,NaN,Two doses should be taken on the first day of ...,NaN,NaN,NaN,NaN,"On subsequent days, doses should be about 12 h...",Relenza®,"[Influenza, Relenza®]",Relenza®


# EVALUATION

In [17]:
df['eval'] = 0
df.loc[df.prescription == df.predmed, 'eval'] = 1
df['eval']
print ("x",df.prescription[1],"x")
print ("x",df.predmed[1],"x")

x Lamisil x
x Lamisil x


In [18]:
score = df['eval'].sum()/df['eval'].count()
print(score*100)

92.15686274509804


# Recommender

In [19]:
listx = ['drug1','drug2','drug3']
for prescribed in listx:
    df[listx] = df[listx].fillna('')
df['therapy'] = df.apply(lambda x: list([x['CONDITION'], x['drug1'],x['drug2'],x['drug3'].strip()]),axis=1)
df['therapy']

0      [Common Cold, Cetrizine dihydrochloride, Pheny...
1        [Athlete's Foot, Terbinafine Hydrochloride, , ]
2                            [Chicken pox, Acylocir, , ]
3                             [Arthritis, Abatacept, , ]
4          [Influenza, zanamivir, lactose monohydrate, ]
                             ...                        
97           [Vertigo, Dimenhydrinate, Dimenhydrinate, ]
98     [Vitamin D def., Cholecalciferol, Cholecalcife...
99                   [Warts, Cantharidin, Podophyllin, ]
100       [Whooping cough, Azithromycin, Azithromycin, ]
101                                     [dry skin, , , ]
Name: therapy, Length: 102, dtype: object

In [20]:
dx = df['therapy']

In [21]:
te = TransactionEncoder()
te_arydx = te.fit(dx).transform(dx)
trans_dx = pd.DataFrame(te_arydx, columns=te.columns_)
trans_dx.head()

,,Abacavir,Abatacept,Acne,Acute sinusitis,Acyclovir,Acylocir,Allergies,Alzheimer's and other Dementias,Amitriptyline,...,uremia,urinary calculi,ursidiol,urticaria,vaginal cancer,valproic acid,whooping cough,zaleplan,zanamivir,zinc sulfate
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [22]:
frequents = apriori(trans_dx, min_support=0.0001, use_colnames=True)

In [23]:
rules = association_rules(frequents, metric = 'lift')
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))
rules = rules[(rules['consequents_len'] <= 2)]

In [24]:
#Convert frozenset objects into string
rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(list(x))).astype(str)
rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(list(x))).astype(str)

# TESTING

In [25]:
diagnosis = ["HIV/AIDS"]
med = rules_mlxtend['consequents'].loc[(rules_mlxtend['antecedents'].isin(diagnosis))]
med = pd.DataFrame(med, dtype=object)


a = df['CONDITION'].tolist()
char = [',']
med = med[~med['consequents'].isin(a)]
medlist = med.consequents.tolist()

print("You Have diagnosed with",diagnosis ,"\nYou may need  -> ",medlist[0] ," !!!")

You Have diagnosed with ['HIV/AIDS'] 
You may need  ->  Triumeq  !!!


In [26]:
recom = rules['consequents'].loc[(rules['antecedents'].isin(diagnosis))]
recom = pd.DataFrame(recom, dtype = object)
recomlst = recom.consequents.tolist()

In [27]:
print ("Maybe you also need: ", recomlst [1:3])

Maybe you also need:  ['Dolutegravir', 'Lamivudine']


In [106]:
index = ['Before BREAKFAST','Before LUNCH','Before DINNER','After BREAKFAST','After LUNCH','After DINNER']
when = df[index]
when = when.fillna(0)
df['when'] = (when.stack()
               .reset_index(name='val')
               .query('val == 1')
               .groupby('level_0')['level_1']
               .apply(list))
df['when'] = df['when'].astype(str)

In [110]:
txprint = ['BRAND_NAME','FORM','when']
dt = df[df['BRAND_NAME'].str.contains('Plaquenil')]
dt = dt[txprint]
dt

,BRAND_NAME,FORM,when
20,Plaquenil,Pill,"['After BREAKFAST', 'After DINNER']"


In [114]:
print (
"R/",dt.BRAND_NAME.to_string(index=False),',',dt.FORM.to_string(index=False), "\n",
'S/',dt.when.to_string(index=False)
)

R/  Plaquenil ,  Pill 
 S/  ['After BREAKFAST', 'After DINNER']
